In [9]:
import numpy as np
from collections import deque, defaultdict
from copy import deepcopy
from functools import cache

In [2]:
with open('data/21.txt', 'r') as f:
    data = f.read().splitlines()

In [3]:
letters_chain = {
    'A': {
        'A': [''],
        '^': ['<'],
        '>': ['v'],
        'v': ['v<', '<v'],
        '<': ['v<<']
    },
    '^': {
        'A': ['>'],
        '^': [''],
        '>': ['v>', '>v'],
        'v': ['v'],
        '<': ['v<']
    },
    '>': {
        'A': ['^'],
        '^': ['^<', '<^'],
        '>': [''],
        'v': ['<'],
        '<': ['<<']
    },
    'v': {
        'A': ['^>', '>^'],
        '^': ['^'],
        '>': ['>'],
        'v': [''],
        '<': ['<']
    },
    '<': {
        'A': ['>>^'],
        '^': ['>^'],
        '>': ['>>'],
        'v': ['>'],
        '<': [''],
    }
}

In [4]:
def find_path_in_deck(deck, letter_start, letter_end):
    res = []
    y_0, x_0 = np.where(deck==letter_start)
    y_0, x_0 = y_0[0], x_0[0]

    y_1, x_1 = np.where(deck==letter_end)
    y_1, x_1 = y_1[0], x_1[0]

    all_res = set()

    if y_1 < y_0:
        res += ['^'] * (y_0 - y_1)
    if x_1 < x_0:
        res += ['<'] * (x_0 - x_1)
    if x_1 > x_0:
        res += ['>'] * (x_1 - x_0)
    if y_1 > y_0:
        res += ['v'] * (y_1 - y_0)
    all_res.add(''.join(res))

    if letter_start not in '7410A' or letter_end not in '7410A':
        res = []
        if x_1 < x_0:
            if y_1 < y_0:
                all_res.add(''.join(['<'] * (x_0 - x_1) + ['^'] * (y_0 - y_1)))
                all_res.add(''.join(['^'] * (y_0 - y_1) + ['<'] * (x_0 - x_1)))
            elif y_1 > y_0:
                all_res.add(''.join(['<'] * (x_0 - x_1) + ['v'] * (y_1 - y_0)))
                all_res.add(''.join(['v'] * (y_1 - y_0) + ['<'] * (x_0 - x_1)))
        elif x_0 < x_1:
            if y_1 < y_0:
                all_res.add(''.join(['>'] * (x_1 - x_0) + ['^'] * (y_0 - y_1)))
                all_res.add(''.join(['^'] * (y_0 - y_1) + ['>'] * (x_1 - x_0)))
            elif y_1 > y_0:
                all_res.add(''.join(['>'] * (x_1 - x_0) + ['v'] * (y_1 - y_0)))
                all_res.add(''.join(['v'] * (y_1 - y_0) + ['>'] * (x_1 - x_0)))
    return all_res

In [5]:
deck = np.array(([['7','8','9'], ['4','5','6'], ['1','2','3'], ['','0','A']]))

In [6]:
letter_paths = defaultdict(dict)
for a in 'A0123456789':
    for b in 'A0123456789':
        if a == b:
            letter_paths[a][b] = []
        letter_paths[a][b] = [x for x in list(find_path_in_deck(deck, a, b))]

In [7]:
def build_combinations(arrays, current=[], index=0):
    if index == len(arrays):
        return [current]
    results = []
    for value in arrays[index]:
        new_results = build_combinations(arrays, current + [value], index + 1)
        results.extend(new_results)
    return results

In [10]:
@cache
def desribe(code, depth):
    if code[0].isnumeric():
        code = 'A' + code
        moves = [[y + 'A' for y in x] for x in [letter_paths[code[i]][code[i+1]] for i in range(len(code[:-1]))]]
        moves = build_combinations(moves)
    else:
        code = 'A' + code
        moves = [[y + 'A' for y in x] for x in [letters_chain[code[i]][code[i+1]] for i in range(len(code[:-1]))]]
        moves = build_combinations(moves)
    if depth == 0:
        return min([sum(map(len, move)) for move in moves])
    else:
        return min([sum(desribe(curr_code, depth - 1) for curr_code in move) for move in moves])

In [11]:
for i, l in enumerate([2, 25]):
    total_res = 0
    for d in data:
        res = desribe(d, l)
        total_res += res * int(d[:3])
    print(f'Part {i+1}: {total_res}')
    

Part 1: 123096
Part 2: 154517692795352
